In [1]:
#!pip install flair
# reference to flair: https://github.com/flairNLP/flair
!pip3 install google-cloud-vision
#!pip install transformers

     |████████████████████████████████| 370 kB 5.1 MB/s 
     |████████████████████████████████| 111 kB 6.4 MB/s 
     |████████████████████████████████| 45 kB 840 kB/s 
     |████████████████████████████████| 1.1 MB 12.4 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 1.26.3
    Uninstalling google-api-core-1.26.3:
      Successfully uninstalled google-api-core-1.26.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-translate 1.5.0 requires google-api-core[grpc]<2.0.0dev,>=1.6.0, but you have google-api-core 2.5.0 which is incompatible.
google-cloud-language 1.2.0 requires google-api-core[grpc]<2.0.0dev,>=1.6.0, but you have google-api-core 2

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from os import listdir
from os.path import isfile, join
mypath = '/content/drive/MyDrive/validation_images/'
imagefiles = [mypath+f for f in listdir(mypath) if isfile(join(mypath, f))]

In [4]:
imagefiles = imagefiles[1:]
imagefiles

['/content/drive/MyDrive/validation_images/PXL_20220206_174349772.jpg',
 '/content/drive/MyDrive/validation_images/PXL_20220206_174433452.jpg',
 '/content/drive/MyDrive/validation_images/PXL_20220206_174443920.jpg',
 '/content/drive/MyDrive/validation_images/PXL_20220206_174454670.jpg',
 '/content/drive/MyDrive/validation_images/PXL_20220206_174509888.jpg',
 '/content/drive/MyDrive/validation_images/PXL_20220206_174516806.jpg',
 '/content/drive/MyDrive/validation_images/PXL_20220206_174627450.jpg',
 '/content/drive/MyDrive/validation_images/PXL_20220206_185443416.jpg']

In [5]:
import google

In [9]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/content/drive/MyDrive/validation_images/package-buddy-capstone-d9edc5dfd127.json"

In [10]:
def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    #print('Texts:')

    for text in texts:
         #print('\n"{}"'.format(text.description))

         vertices = (['({},{})'.format(vertex.x, vertex.y)
                     for vertex in text.bounding_poly.vertices])

         #print('bounds: {}'.format(','.join(vertices)))
    
    return texts

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

In [11]:
output_list = [detect_text(i) for i in imagefiles]

In [12]:
output_list = [str(i) for i in output_list]

In [13]:
import re
def clear_text(text: str) -> str:
    # Remove emojis
    #text = text.encode('ascii', errors='ignore').decode('ascii')

    #Remove the 'RT's
    #text = re.sub('RT\s', '', text)

    # Remove the links
    #text = re.sub('http\S*|www\S*', '', text)
    

    # Remove the breakline code
    text = text.replace('\\n', ' ')
    

    # Remove the breakline code
    #text = re.sub('\\ ', '', text)

    # Remove the breakline code
    #text = re.sub(' \\ n', '', text)

    # Remove usernames 
    #text = re.sub('\@\S*', '', text)

    # Remove hashtags
    # text = re.sub('#\S*', '', text)

    # Remove numbers
    #text = re.sub('[0-9]+', '', text)

    # Split before a word
    text = text.split("bounding")[0]
    text = text.split("locale")[1]
    
    
    

    # Remove punctuation and extra blank spaces
    text = re.sub('\W+', ' ', text)
    

    
    text = re.sub("[0-9][A-Z][a-z]+", "", text)
    
    # Remove stop words and words with 2 or fewer letters
    #text = [word for word in text.split() if 
    #        (len(word) >= 3) and (word not in stop_words)]

    #text = ' '.join(text).lower()

    return text

In [14]:
output_list[0]

'[locale: "en"\ndescription: "135 EF\\nlizeriz\\304\\261u\\304\\261\\nYYZ1\\n4.9Lbs\\nDT03\\nGeraldine Mercado\\n112George Street\\nSUITE S533\\nM5A 2M5 Toronto, Ontario Canada\\nTBC369899292009\\nCYCLE 1\\nYYZ4\\nO T03\\nYHM5\\nC ART\\n"\nbounding_poly {\n  vertices {\n    x: 390\n    y: 395\n  }\n  vertices {\n    x: 3591\n    y: 395\n  }\n  vertices {\n    x: 3591\n    y: 2688\n  }\n  vertices {\n    x: 390\n    y: 2688\n  }\n}\n, description: "135"\nbounding_poly {\n  vertices {\n    x: 2985\n    y: 395\n  }\n  vertices {\n    x: 3065\n    y: 396\n  }\n  vertices {\n    x: 3065\n    y: 417\n  }\n  vertices {\n    x: 2985\n    y: 416\n  }\n}\n, description: "EF"\nbounding_poly {\n  vertices {\n    x: 3108\n    y: 397\n  }\n  vertices {\n    x: 3153\n    y: 398\n  }\n  vertices {\n    x: 3153\n    y: 419\n  }\n  vertices {\n    x: 3108\n    y: 418\n  }\n}\n, description: "lizeriz\\304\\261u\\304\\261"\nbounding_poly {\n  vertices {\n    x: 2954\n    y: 423\n  }\n  vertices {\n    x: 

In [18]:
list1 = []

for i in output_list:
    x = clear_text(i)
    list1.append(x)
#print(list1)
list1[7]

' en description REAKBULK entiro Sy stems v 1 1 16 1605 BBX HL Contact FROM Mango Barcelona Origin Mercaders 9 11 Pol Ind Riera de Caldes 08184 Palau Solit Plegamans SPAIN BCN TO leah travaglini 2008 112 george street Contact 4169995948 M5A 2M5 TORONTO Canada CA YHM YTZ Day Time Ret code Pickup date Package Shpt Wght Piece CYD08J00 0 20kg 0 20kg 1 1 2022 02 01 Content Clothes Cleared under waybill WAYBILL 98 4285 7765 2L 303 207AM5A2M5 41000000 00 38427907 500553542 9 SAP 3001 101 PED 00 38427907500553542 Canada leah travaglini 01 02 22 T Clent sig nature CYD08J00 '

In [20]:
for i in list1:
  print(i)

 en description 135 EF lizeriz 304 261u 304 261 YYZ1 4  DT03 Geraldine Mercado 11 Street SUITE S533 M5A 2M5 Toronto Ontario Canada TBC369899292009 CYCLE 1 YYZ4 O T03 YHM5 C ART 
 en description YYZ1 1 6 Lbs 02 06 DT03 Alison Roy 1503 112 George Street M5A 2M5 Toronto Ontario Canada Not restricted as per special provision A199 TBC370081708009 DT03 CYCLE 1 YYZ7 O T03 YHM5 C ART livr Is MhmbHHtj 
 en description GIES in M7vxB3 1 404 based Dia riqu 303 251e 303 240 par e v 303 251g 303 251tale orn size 6 n 303 251 talle 6 Ou Sont mounl 303 251es simplement avec une d al humedecerse Si observa una peque 303 261a on una toallita para beb 303 251s o un pa 303 261o h 303 272medo un keep plastic bags away from babies a to their mouths To avoid risk of choking e most articles of clothing HUGGIES 302 256 dia INGE8XAL 1 404 IMPORTANT Pour 303 251viter tout risque de ATTENTION Les enfants peu un b 303 251b 303 251 d 303 251chirer lac risquent de of flame 22 YYZ1 5 5 Lbs 02 06 DT03 Nell Chitty N711 

In [ ]:
list1[10][48]

'0'

In [ ]:
len(output_list)

83

In [ ]:
len(list1)

83

In [ ]:
# from transformers import AutoModelForTokenClassification, AutoTokenizer
# import torch

# model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
# tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# label_list = [
#     "O",       # Outside of a named entity
#     "B-MISC",  # Beginning of a miscellaneous entity right after another miscellaneous entity
#     "I-MISC",  # Miscellaneous entity
#     "B-PER",   # Beginning of a person's name right after another person's name
#     "I-PER",   # Person's name
#     "B-ORG",   # Beginning of an organisation right after another organisation
#     "I-ORG",   # Organisation
#     "B-LOC",   # Beginning of a location right after another location
#     "I-LOC"    # Location
# ]

# sequence = list1[0]

# # Bit of a hack to get the tokens with the special tokens
# tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
# inputs = tokenizer.encode(sequence, return_tensors="pt")

# outputs = model(inputs)[0]
# predictions = torch.argmax(outputs, dim=2)

# print([(token, label_list[prediction]) for token, prediction in zip(tokens, predictions[0].tolist())])

In [ ]:
# from transformers import AutoModelForTokenClassification, AutoTokenizer
# import torch

# model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
# tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# label_list = [
#     "O",       # Outside of a named entity
#     "B-MISC",  # Beginning of a miscellaneous entity right after another miscellaneous entity
#     "I-MISC",  # Miscellaneous entity
#     "B-PER",   # Beginning of a person's name right after another person's name
#     "I-PER",   # Person's name
#     "B-ORG",   # Beginning of an organisation right after another organisation
#     "I-ORG",   # Organisation
#     "B-LOC",   # Beginning of a location right after another location
#     "I-LOC"    # Location
# ]

# sequence = list1[0]

# # Bit of a hack to get the tokens with the special tokens
# tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
# inputs = tokenizer.encode(sequence, return_tensors="pt")

# outputs = model(inputs)[0]
# predictions = torch.argmax(outputs, dim=2)

# print([(token, label_list[prediction]) for token, prediction in zip(tokens, predictions[0].tolist())])

In [ ]:
# from flair.data import Sentence
# from flair.models import SequenceTagger

# # make a sentence
# sentence = []
# for i in list1:
#     x = Sentence(i)
#     sentence.append(x)



# # load the NER tagger
# tagger = SequenceTagger.load('ner')

# # run NER over sentence
# tagger.predict(sentence)

In [ ]:
# sentence

In [ ]:
# print(sentence)
# print('The following NER tags are found:')

# # iterate over entities and print
# for entity in Sentence.get_spans('ner'):
#     print(entity)

In [ ]:
# !pip install spacy

In [ ]:
# import spacy
# from spacy.tokens import DocBin
# import pandas as pd
# import re
# pd.set_option('display.max_colwidth', -1)

# def massage_data(address):
#     '''Pre process address string to remove new line characters, add comma punctuations etc.'''
#     cleansed_address1=re.sub(r'(,)(?!\s)',', ',address)
#     cleansed_address2=re.sub(r'(\\n)',', ',cleansed_address1)
#     cleansed_address3=re.sub(r'(?!\s)(-)(?!\s)',' - ',cleansed_address2)
#     cleansed_address=re.sub(r'\.','',cleansed_address3)
#     return cleansed_address

# def get_address_span(address=None,address_component=None,label=None):
#     '''Search for specified address component and get the span.
#     Eg: get_address_span(address="221 B, Baker Street, London",address_component="221",label="BUILDING_NO") would return (0,2,"BUILDING_NO")'''

#     if pd.isna(address_component) or str(address_component)=='nan':
#         pass
#     else:
#         address_component1=re.sub('\.','',address_component)
#         address_component2=re.sub(r'(?!\s)(-)(?!\s)',' - ',address_component1)
#         span=re.search('\\b(?:'+address_component2+')\\b',address)
#         #print(span)
#         return (span.start(),span.end(),label)

# def extend_list(entity_list,entity):
#     if pd.isna(entity):
#         return entity_list
#     else:
#         entity_list.append(entity)
#         return entity_list

# def create_entity_spans(df,tag_list):
#     '''Create entity spans for training/test datasets'''
#     df['Address']=df['Address'].apply(lambda x: massage_data(x))
#     df["BuildingTag"]=df.apply(lambda row:get_address_span(address=row['Address'],address_component=row['Building_Name'],label='BUILDING_NAME'),axis=1)
#     df["BuildingNoTag"]=df.apply(lambda row:get_address_span(address=row['Address'],address_component=row['Building_Number'],label='BUILDING_NO'),axis=1)
#     df["RecipientTag"]=df.apply(lambda row:get_address_span(address=row['Address'],address_component=row['Recipient'],label='RECIPIENT'),axis=1)
#     df["StreetNameTag"]=df.apply(lambda row:get_address_span(address=row['Address'],address_component=row['Street_Name'],label='STREET_NAME'),axis=1)
#     df["ZipCodeTag"]=df.apply(lambda row:get_address_span(address=row['Address'],address_component=row['Zip_Code'],label='ZIP_CODE'),axis=1)
#     df["CityTag"]=df.apply(lambda row:get_address_span(address=row['Address'],address_component=row['City'],label='CITY'),axis=1)
#     df["CountryTag"]=df.apply(lambda row:get_address_span(address=row['Address'],address_component=row['Country'],label='COUNTRY'),axis=1)
#     df["StateTag"]=df.apply(lambda row:get_address_span(address=row['Address'],address_component=row['State'],label='STATE'),axis=1)
#     df['EmptySpan']=df.apply(lambda x: [], axis=1)

#     for i in tag_list:
#         df['EntitySpans']=df.apply(lambda row: extend_list(row['EmptySpan'],row[i]),axis=1)
#         df['EntitySpans']=df[['EntitySpans','Address']].apply(lambda x: (x[1], x[0]),axis=1)
#     return df['EntitySpans']

# def get_doc_bin(training_data,nlp):
#     '''Create DocBin object for building training/test corpus'''
#     # the DocBin will store the example documents
#     db = DocBin()
#     for text, annotations in training_data:
#         doc = nlp(text) #Construct a Doc object
#         ents = []
#         for start, end, label in annotations:
#             span = doc.char_span(start, end, label=label)
#             ents.append(span)
#         doc.ents = ents
#         db.add(doc)
#     return db

# #Load blank English model. This is needed for initializing a Document object for our training/test set.
# nlp = spacy.blank("en")

# #Define custom entity tag list
# tag_list=["BuildingTag","BuildingNoTag","RecipientTag","StreetNameTag","ZipCodeTag","CityTag","StateTag","CountryTag"]

# ###### Training dataset prep ###########
# # Read the training dataset into pandas
# df_train=pd.read_csv(filepath_or_buffer="./corpus/dataset/us-train-dataset.csv",sep=",",dtype=str)

# # Get entity spans
# df_entity_spans= create_entity_spans(df_train.astype(str),tag_list)
# training_data= df_entity_spans.values.tolist()

# # Get & Persist DocBin to disk
# doc_bin_train= get_doc_bin(training_data,nlp)
# doc_bin_train.to_disk("./corpus/spacy-docbins/train.spacy")
# ######################################


# ###### Validation dataset prep ###########
# # Read the validation dataset into pandas
# df_test=pd.read_csv(filepath_or_buffer="./corpus/dataset/us-test-dataset.csv",sep=",",dtype=str)

# # Get entity spans
# df_entity_spans= create_entity_spans(df_test.astype(str),tag_list)
# validation_data= df_entity_spans.values.tolist()

# # Get & Persist DocBin to disk
# doc_bin_test= get_doc_bin(validation_data,nlp)
# doc_bin_test.to_disk("./corpus/spacy-docbins/test.spacy")
# ##########################################

In [ ]:
#sentence[:][:]

In [ ]:
import spacy
from spacy.tokens import DocBin
import pandas as pd
import re
pd.set_option('display.max_colwidth', None)

In [ ]:
def massage_data(sentence):
  '''Pre process address string to remove new line characters, add comma punctuations etc.'''
  cleansed_sentence1=re.sub(r'(,)(?!\s)',', ',sentence)
  cleansed_sentence2=re.sub(r'(\\n)',', ',cleansed_sentence1)
  cleansed_sentence3=re.sub(r'(?!\s)(-)(?!\s)',' - ',cleansed_sentence2)
  cleansed_sentence=re.sub(r'\.','',cleansed_sentence3)
  return(cleansed_sentence)

In [ ]:
# list2 = []

# for i in output_list:
#     x = massage_data(i)
#     list2.append(x)
# #print(list1)
# list2[9]

In [ ]:
# from flair.data import Sentence
# from flair.models import SequenceTagger

# # make a sentence
# sentence2 = []
# for i in list2:
#     x = Sentence(i)
#     sentence2.append(x)



# # load the NER tagger
# tagger = SequenceTagger.load('ner')

# # run NER over sentence
# tagger.predict(sentence2)

In [ ]:
#list1

In [ ]:
# sentence[0][:]

In [ ]:
list2 = []

for i in list1:
    x = massage_data(i)
    list2.append(x)
#print(list1)
list2[9]

' en description CANADA POST POSTES CANADA TM Expedited Parcel Colis acc 303 251l 303 251r 303 251s TO A Tel Tel 800 832 7569 CYNTHIA INNES CYNTHIA INNES 116 GEORGE STREET APT 1506 TORONTO ON M5A 3S2 M5A 3S2 TRACKING NUMBER 7036 7330 5514 9304 N 302 260 DE REP 303 211RAGE Sender warrants that this item does not contain non mailable matter L exp 303 251diteur garantit que cet envoi ne contient pas d objet inadmissible ereuses FROM DE INDIGO BOOKS MUSIC CDF BLDG 50 100 ALFRED KUEHNE BLVD BRAMPTON ON L6T 4K4 1 00 KG VE EV MANIFEST REQ MANIFESTE REQ P F 7036733 A C VIN NIF 019 SPEC 3696 V2 0 Reference No 657182164 '

In [ ]:
#list2

In [ ]:
new_list = [massage_data(i) for i in list2]
new_list[0]

' en description YYZ1 1 1 Lbs 01 23 DT03 Leslie Walsh 626 112 George Street M5A 2M5 Toronto Ontarlo Canada TBC360938832009 DT03 CYCLE 1 YYZ9 T03 MWSrgRSF5 MW5 9RSF5 A 20 3 INO '

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(list2[0:-1])

In [ ]:
df_train = pd.DataFrame(list2[0:50])
df_test = pd.DataFrame(list2[50:])


In [ ]:
df_train[0] = df_train[0].astype(str)
print(df_train.dtypes)
df_test[0] = df_test[0].astype(str)
print(df_test.dtypes)

0    object
dtype: object
0    object
dtype: object


In [ ]:
df.rename(columns={0: 'Address'}, inplace=True)

In [ ]:
csv_data = df.to_csv(index=False)

In [ ]:
csv_data

'Address\n en description YYZ1 1 1 Lbs 01 23 DT03 Leslie Walsh 626 112 George Street M5A 2M5 Toronto Ontarlo Canada TBC360938832009 DT03 CYCLE 1 YYZ9 T03 MWSrgRSF5 MW5 9RSF5 A 20 3 INO \n en description CANADA POSTES POST 12 Expedited Parcel TM Colis acc 303 251l 303 251r 303 251g CANADA TO Tel TI BRITTANY DHAMALIE 112 GEORGE ST S628 TORONTO ON M5A 2M5 18555576844 320 2345 320 220 2 320 2345 20 3 x 13 3 x880 CU KG VE EU d object inadmissible S1 REQ \n en description YYZ1 0 4 Lbs 01 23 DT03 Daniel Trigiani 1208 112 George Street pri M5A 2M5 Toronto Ontario Canada TBC361089405009 DT03 CYCLE 1 cit 303 251 sans MARR ee YYZ7 O TO3 YHM5 C ART M7apBWS85 1 801 M7DPBWS85 1 801 \n en description 1 23 22 5 57 PM Production Sheet G70 TORFS1 GC Toronto WOW 1 Order ID 25399741 Wow Totes 1 1 Alexandra Name Sukhareva Address 112 George Street App S617 Toronto Ontario M5A2M5 Phone 647 991 2926 320 242 321 203 321 200 320 265 App Apt 2022 01 23 Created at 17 40 29 Special Instructions Delivery Day Sunda

In [ ]:
df_train.rename(columns={0: 'Address'}, inplace=True)
df_test.rename(columns={0: 'Address'}, inplace=True)

In [ ]:
df_train.head()

,Address
0,en description YYZ1 1 1 Lbs 01 23 DT03 Leslie Walsh 626 112 George Street M5A 2M5 Toronto Ontarlo Canada TBC360938832009 DT03 CYCLE 1 YYZ9 T03 MWSrgRSF5 MW5 9RSF5 A 20 3 INO
1,en description CANADA POSTES POST 12 Expedited Parcel TM Colis acc 303 251l 303 251r 303 251g CANADA TO Tel TI BRITTANY DHAMALIE 112 GEORGE ST S628 TORONTO ON M5A 2M5 18555576844 320 2345 320 220 2 320 2345 20 3 x 13 3 x880 CU KG VE EU d object inadmissible S1 REQ
2,en description YYZ1 0 4 Lbs 01 23 DT03 Daniel Trigiani 1208 112 George Street pri M5A 2M5 Toronto Ontario Canada TBC361089405009 DT03 CYCLE 1 cit 303 251 sans MARR ee YYZ7 O TO3 YHM5 C ART M7apBWS85 1 801 M7DPBWS85 1 801
3,en description 1 23 22 5 57 PM Production Sheet G70 TORFS1 GC Toronto WOW 1 Order ID 25399741 Wow Totes 1 1 Alexandra Name Sukhareva Address 112 George Street App S617 Toronto Ontario M5A2M5 Phone 647 991 2926 320 242 321 203 321 200 320 265 App Apt 2022 01 23 Created at 17 40 29 Special Instructions Delivery Day Sunday Delivery Date 2022 01 23 Route TBD Sequence 00 25399741
4,en description YYZ1 1 0 Lbs 01 23 DT03 Don Ho mes 112 George Street Suite 1 06 M5A 2N5 Toronto Ontario Canada TBC360972961009 DT03 CYCLE 1 YYZ4 O T03 YHM5 C ART spNi1v Mvf9C4SS5 1 3022 Mivi9C4SS5


In [ ]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.0 MB 8.4 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [ ]:
import spacy
from spacy import displacy 
nlp = spacy.load('en_core_web_sm')
# Text with nlp
doc = nlp(list2[3])
# Display Entities
displacy.render(doc, style="ent")

'<div class="entities" style="line-height: 2.5; direction: ltr"> en description \n<mark class="entity" style="background: #e4e7d2; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">\n    1\n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; text-transform: uppercase; vertical-align: middle; margin-left: 0.5rem">CARDINAL</span>\n</mark>\n 23 22 \n<mark class="entity" style="background: #bfe1d9; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">\n    5 57 PM\n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; text-transform: uppercase; vertical-align: middle; margin-left: 0.5rem">TIME</span>\n</mark>\n \n<mark class="entity" style="background: #ffeb80; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">\n    Production Sheet G70\n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0

In [ ]:
import spacy
from spacy.tokens import DocBin
import pandas as pd
import re
pd.set_option('display.max_colwidth', None)

def massage_data(address):
    '''Pre process address string to remove new line characters, add comma punctuations etc.'''
    cleansed_address1=re.sub(r'(,)(?!\s)',', ',address)
    cleansed_address2=re.sub(r'(\\n)',', ',cleansed_address1)
    cleansed_address3=re.sub(r'(?!\s)(-)(?!\s)',' - ',cleansed_address2)
    cleansed_address=re.sub(r'\.','',cleansed_address3)
    return cleansed_address

def get_address_span(address=None,address_component=None,label=None):
    '''Search for specified address component and get the span.
    Eg: get_address_span(address="221 B, Baker Street, London",address_component="221",label="BUILDING_NO") would return (0,2,"BUILDING_NO")'''

    if pd.isna(address_component) or str(address_component)=='nan':
        pass
    else:
        address_component1=re.sub('\.','',address_component)
        address_component2=re.sub(r'(?!\s)(-)(?!\s)',' - ',address_component1)
        span=re.search('\\b(?:'+address_component2+')\\b',address)
        #print(span)
        return (span.start(),span.end(),label)

def extend_list(entity_list,entity):
    if pd.isna(entity):
        return entity_list
    else:
        entity_list.append(entity)
        return entity_list

In [ ]:
from transformers import AutoModelForTokenClassification,AutoTokenizer
import torch


model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/998 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [ ]:
label_list = [
    "O",       # Outside of a named entity
    "B-MISC",  # Beginning of a miscellaneous entity right after another miscellaneous entity
    "I-MISC",  # Miscellaneous entity
    "B-PER",   # Beginning of a person's name right after another person's name
    "I-PER",   # Person's name
    "B-ORG",   # Beginning of an organisation right after another organisation
    "I-ORG",   # Organisation
    "B-LOC",   # Beginning of a location right after another location
    "I-LOC"    # Location
]

sequence = list2[0]

# Bit of a hack to get the tokens with the special tokens
tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
inputs = tokenizer.encode(sequence, return_tensors="pt")

outputs = model(inputs)[0]
predictions = torch.argmax(outputs, dim=2)

print([(token, label_list[prediction]) for token, prediction in zip(tokens, predictions[0].tolist())])

[('[CLS]', 'O'), ('en', 'O'), ('description', 'O'), ('Y', 'O'), ('##Y', 'O'), ('##Z', 'O'), ('##1', 'O'), ('1', 'O'), ('1', 'O'), ('L', 'O'), ('##bs', 'O'), ('01', 'O'), ('23', 'O'), ('D', 'O'), ('##T', 'O'), ('##0', 'O'), ('##3', 'O'), ('Leslie', 'I-ORG'), ('Walsh', 'I-ORG'), ('62', 'O'), ('##6', 'O'), ('112', 'O'), ('George', 'I-LOC'), ('Street', 'I-LOC'), ('M', 'O'), ('##5', 'O'), ('##A', 'O'), ('2', 'O'), ('##M', 'O'), ('##5', 'O'), ('Toronto', 'I-ORG'), ('On', 'I-ORG'), ('##tar', 'I-ORG'), ('##lo', 'I-ORG'), ('Canada', 'I-ORG'), ('T', 'I-ORG'), ('##BC', 'I-ORG'), ('##36', 'O'), ('##0', 'O'), ('##9', 'O'), ('##38', 'O'), ('##8', 'O'), ('##32', 'O'), ('##00', 'O'), ('##9', 'O'), ('D', 'O'), ('##T', 'O'), ('##0', 'O'), ('##3', 'O'), ('C', 'O'), ('##Y', 'O'), ('##CL', 'O'), ('##E', 'O'), ('1', 'O'), ('Y', 'O'), ('##Y', 'O'), ('##Z', 'O'), ('##9', 'O'), ('T', 'O'), ('##0', 'O'), ('##3', 'O'), ('MW', 'O'), ('##S', 'O'), ('##rg', 'O'), ('##RS', 'O'), ('##F', 'O'), ('##5', 'O'), ('MW', 'O

In [ ]:
sequence = new_list[0]
tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))

inputs = tokenizer.encode(sequence, return_tensors="pt")

outputs = model(inputs)[0]
predictions = torch.argmax(outputs, dim=2)

# print([(token, label_list[prediction]) for token, prediction in zip(tokens, predictions[0].tolist())])
List_Of_Full_Names = []
List_of_names = []
Last_index = -1
for i in range(0, predictions.shape[1]):
  if predictions[0,i] == 3 or predictions[0,i] == 4:
      List_of_names.append(tokens[i])
  else:
    if List_of_names !=[]:
      List_Of_Full_Names.append(List_of_names)
    Last_index = -1
    List_of_names = []
Final_Names = []
strname = ''
replaced_name = ''
for i in range(0, len(List_Of_Full_Names)):
  if List_Of_Full_Names[i] !=[]:
    for ii in List_Of_Full_Names[i]:
      if '#' in ii:
        replaced_name = re.sub('#','',ii)
      else:
        replaced_name = ii
      for chars in replaced_name:
        if chars.isupper():
          strname+=" "
      strname+=replaced_name
print(strname)

In [ ]:
List_Of_Full_Names


[]

In [ ]:
def create_entity_spans(df,tag_list):
    '''Create entity spans for training/test datasets'''
    df['Address']=df['Address'].apply(lambda x: massage_data(x))
    df["BuildingTag"]=df.apply(lambda row:get_address_span(address=row['Address']),axis=1)
    df["BuildingNoTag"]=df.apply(lambda row:get_address_span(address=row['Address']),axis=1)
    df["RecipientTag"]=df.apply(lambda row:get_address_span(address=row['Address']),axis=1)
    df["StreetNameTag"]=df.apply(lambda row:get_address_span(address=row['Address']),axis=1)
    df["ZipCodeTag"]=df.apply(lambda row:get_address_span(address=row['Address']),axis=1)
    df["CityTag"]=df.apply(lambda row:get_address_span(address=row['Address']),axis=1)
    df["CountryTag"]=df.apply(lambda row:get_address_span(address=row['Address']),axis=1)
    df["StateTag"]=df.apply(lambda row:get_address_span(address=row['Address']),axis=1)
    df['EmptySpan']=df.apply(lambda x: [], axis=1)

    for i in tag_list:
        df['EntitySpans']=df.apply(lambda row: extend_list(row['EmptySpan'],row[i]),axis=1)
        df['EntitySpans']=df[['EntitySpans','Address']].apply(lambda x: (x[1], x[0]),axis=1)
    return df['EntitySpans']

In [ ]:
def get_doc_bin(training_data,nlp):
    '''Create DocBin object for building training/test corpus'''
    # the DocBin will store the example documents
    db = DocBin()
    for text, annotations in training_data:
        doc = nlp(text) #Construct a Doc object
        ents = []
        for start, end, label in annotations:
            span = doc.char_span(start, end, label=label)
            ents.append(span)
        doc.ents = ents
        db.add(doc)
    return db

In [ ]:
#Load blank English model. This is needed for initializing a Document object for our training/test set.
nlp = spacy.blank("en")

#Define custom entity tag list
tag_list=["BuildingTag","BuildingNoTag","RecipientTag","StreetNameTag","ZipCodeTag","CityTag","StateTag","CountryTag"]

###### Training dataset prep ###########
# Read the training dataset into pandas
df_train

# Get entity spans
df_entity_spans= create_entity_spans(df_train.astype(str),tag_list)
training_data= df_entity_spans.values.tolist()

In [ ]:
!pip install spacy==3.0.6

     |████████████████████████████████| 12.8 MB 7.9 MB/s 
     |████████████████████████████████| 9.1 MB 22.3 MB/s 
     |████████████████████████████████| 451 kB 64.0 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 628 kB 37.1 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [ ]:
# Get & Persist DocBin to disk
doc_bin_train= get_doc_bin(training_data,nlp)
doc_bin_train.to_disk("./corpus/spacy-docbins/train.spacy")
######################################


###### Validation dataset prep ###########
# Read the validation dataset into pandas
df_test

# Get entity spans
df_entity_spans= create_entity_spans(df_test.astype(str),tag_list)
validation_data= df_entity_spans.values.tolist()

# Get & Persist DocBin to disk
doc_bin_test= get_doc_bin(validation_data,nlp)
doc_bin_test.to_disk("./corpus/spacy-docbins/test.spacy")
##########################################

AttributeError: ignored

In [ ]:
import spacy

nlp=spacy.load("output\models\model-best")

address_list=["130 W BOSE ST STE 100, PARK RIDGE, IL, 60068, USA",
              "8311 MCDONALD RD, HOUSTON, TX, 77053-4821, USA",
              "PO Box 317, 4100 Hwy 20 E Ste 403, NICEVILLE, FL, 32578-5037, USA",
              "C/O Elon Musk Innovations Inc, 1548 E Florida Avenue, Suite 209, TAMPA, FL, 33613, USA",
              "Seven Edgeway Plaza, C/O Mac Dermott Inc, OAKBROOK TERRACE, IL, 60181, USA"]

# Checking predictions for the NER model
for address in address_list:
    doc=nlp(address)
    ent_list=[(ent.text, ent.label_) for ent in doc.ents]
    print("Address string -> "+address)
    print("Parsed address -> "+str(ent_list))
    print("******")



address="C/o John Doe LLC, 111 8th Avenue Ste 1301, Tulsa, Oklahoma, 74136–1922, USA"
doc=nlp(address)
ent_list=[(ent.text, ent.label_) for ent in doc.ents]
print("Address string -> "+address)
print("Parsed address -> "+str(ent_list))

# Loading Entity Ruler coupled NER model and checking prediction
nlp=spacy.load("output\models_er\model-best")

doc=nlp(address)
ent_list=[(ent.text, ent.label_) for ent in doc.ents]
print("Address string -> "+address)
print("Parsed address -> "+str(ent_list))

OSError: ignored

In [ ]:
def countValidWords(self, sentence: str) -> int:
       
       # parse and get each word from sentence
       words = sentence.split()
       
       # regular expression pattern for valid words
       pattern = re.compile( r'^([a-zA-Z]+\-?[a-z]+[!\.,]?)$|^([a-z]*[!\.,]?)$' )
       
       # valid word count
       count = 0
       
       # scan each word from word pool
       for word in words:
           
           # judge whether current word is valid or not
           match = re.match(pattern, word)
           
           if match:
               count+=1
               
       return words

In [ ]:
def countValidWords(self, sentence: str) -> int:
       
       # parse and get each word from sentence
       words = sentence.split()
       
       # regular expression pattern for valid words
       pattern = re.compile( r'^([a-z]+\-?[a-z]+[!\.,]?)$|^([a-z]*[!\.,]?)$' )
       
       # valid word count
       count = 0
       
       # scan each word from word pool
       for word in words:
           
           # judge whether current word is valid or not
           match = re.match(pattern, word)
           
           if match:
               count+=1
               
       return count

In [ ]:
r = re.compile(r'([a-zA-Z][a-zA-Z0-9][a-z])?')
newlist = list(filter(r.match, new_list)) # Read Note
print(newlist)
len(newlist)

[' en description YYZ1 1 1 Lbs 01 23 DT03 Leslie Walsh 626 112 George Street M5A 2M5 Toronto Ontarlo Canada TBC360938832009 DT03 CYCLE 1 YYZ9 T03 MWSrgRSF5 MW5 9RSF5 A 20 3 INO ', ' en description CANADA POSTES POST 12 Expedited Parcel TM Colis acc 303 251l 303 251r 303 251g CANADA TO Tel TI BRITTANY DHAMALIE 112 GEORGE ST S628 TORONTO ON M5A 2M5 18555576844 320 2345 320 220 2 320 2345 20 3 x 13 3 x880 CU KG VE EU d object inadmissible S1 REQ ', ' en description YYZ1 0 4 Lbs 01 23 DT03 Daniel Trigiani 1208 112 George Street pri M5A 2M5 Toronto Ontario Canada TBC361089405009 DT03 CYCLE 1 cit 303 251 sans MARR ee YYZ7 O TO3 YHM5 C ART M7apBWS85 1 801 M7DPBWS85 1 801 ', ' en description 1 23 22 5 57 PM Production Sheet G70 TORFS1 GC Toronto WOW 1 Order ID 25399741 Wow Totes 1 1 Alexandra Name Sukhareva Address 112 George Street App S617 Toronto Ontario M5A2M5 Phone 647 991 2926 320 242 321 203 321 200 320 265 App Apt 2022 01 23 Created at 17 40 29 Special Instructions Delivery Day Sunday 

83

In [ ]:
pattern = re.compile( r'^([a-zA-Z]+[a-zA-Z0-9]?+([a-z])?' )
if any((match = pattern.match(x)) for x in new_list):
  print(match.group(0))

SyntaxError: ignored

In [ ]:
def clear_text(text: str) -> str:
    pattern = re.compile( r'^([a-z]+[a-zA-Z0-9]?+([a-z])?' )
    # # Remove emojis
    # #text = text.encode('ascii', errors='ignore').decode('ascii')
    words = text.split()

    # #Remove the 'RT's
    # #text = re.sub('RT\s', '', text)
    

    # # Remove the links
    # #text = re.sub('http\S*|www\S*', '', text)
    

    # # Remove the breakline code
    # text = text.replace('\\n', ' ')
    

    # # Remove the breakline code
    # #text = re.sub('\\ ', '', text)

    # # Remove the breakline code
    # #text = re.sub(' \\ n', '', text)

    # # Remove usernames 
    # #text = re.sub('\@\S*', '', text)

    # # Remove hashtags
    # # text = re.sub('#\S*', '', text)

    # # Remove numbers
    # #text = re.sub('[0-9]+', '', text)

    # # Split before a word
    # text = text.split("bounding")[0]
    # text = text.split("locale")[1]
    
    
    

    # # Remove punctuation and extra blank spaces
    # text = re.sub('\W+', ' ', text)
    
    words = words.lower()
    for word in words:
           
           # judge whether current word is valid or not
           match = re.match(pattern, word)
    
    # Remove stop words and words with 2 or fewer letters
    #text = [word for word in text.split() if 
    #        (len(word) >= 3) and (word not in stop_words)]

    #text = ' '.join(text).lower()

    return text

In [ ]:
# import numpy as np


# csv_file  =np.savetxt("shows.csv", new_list, delimiter=", ", fmt="% s")

In [ ]:
# print(csv_file)

In [ ]:
nl = []

for i in new_list[0][0]:
    x = clear_text(i)
    nl.append(x)
#print(list1)
nl[10]

error: ignored

In [ ]:
new_list

[' en description YYZ1 1 1 Lbs 01 23 DT03 Leslie Walsh 626 112 George Street M5A 2M5 Toronto Ontarlo Canada TBC360938832009 DT03 CYCLE 1 YYZ9 T03 MWSrgRSF5 MW5 9RSF5 A 20 3 INO ',
 ' en description CANADA POSTES POST 12 Expedited Parcel TM Colis acc 303 251l 303 251r 303 251g CANADA TO Tel TI BRITTANY DHAMALIE 112 GEORGE ST S628 TORONTO ON M5A 2M5 18555576844 320 2345 320 220 2 320 2345 20 3 x 13 3 x880 CU KG VE EU d object inadmissible S1 REQ ',
 ' en description YYZ1 0 4 Lbs 01 23 DT03 Daniel Trigiani 1208 112 George Street pri M5A 2M5 Toronto Ontario Canada TBC361089405009 DT03 CYCLE 1 cit 303 251 sans MARR ee YYZ7 O TO3 YHM5 C ART M7apBWS85 1 801 M7DPBWS85 1 801 ',
 ' en description 1 23 22 5 57 PM Production Sheet G70 TORFS1 GC Toronto WOW 1 Order ID 25399741 Wow Totes 1 1 Alexandra Name Sukhareva Address 112 George Street App S617 Toronto Ontario M5A2M5 Phone 647 991 2926 320 242 321 203 321 200 320 265 App Apt 2022 01 23 Created at 17 40 29 Special Instructions Delivery Day Sund